In [1]:
#Smiles toxicity classification
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras 
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import rdkit
from rdkit import Chem

In [3]:
#Dataset 1
df_train_smiles=pd.read_csv("smiles_train.csv")
df_test_smiles=pd.read_csv("smiles_test.csv")
df_train_labels=pd.read_csv("y_train.csv")
df_test_labels=pd.read_csv("y_test.csv")


smiles_train=df_train_smiles['smiles']
smiles_test=df_test_smiles['smiles']
y_train=df_train_labels['labels']
y_test=df_test_labels['labels']

In [5]:
#https://iwatobipen.wordpress.com/2017/01/22/encode-and-decode-smiles-strings/

SMILES_CHARS = [' ',
                  '#', '%', '(', ')', '+', '-', '.', '/',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '=', '@',
                  'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                  'R', 'S', 'T', 'V', 'X', 'Z',
                  '[', '\\', ']',
                  'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                  't', 'u',':','d','G','h','W','U','D','y','Y']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )
def smiles_encoder( smiles, maxlen=1000 ):
    smiles =smiles
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c] ] = 1
    return X
 
def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[ i ]
    return smi

In [12]:
X_train=[smiles_encoder(i) for i in smiles_train]
X_train=np.array(X_train)

X_test=[smiles_encoder(i) for i in smiles_test]
X_test=np.array(X_test)

X=[]

for i in np.arange(0,X_train.shape[0]):
    X.append(X_train[i])
    
for i in np.arange(0,X_test.shape[0]):
    X.append(X_test[i])
X=np.array(X)
    
y=[]    

for i in np.arange(0,y_train.shape[0]):
    y.append(y_train[i])
    
for i in np.arange(0,y_test.shape[0]):
    y.append(y_test[i])
    
y=np.array(y)    

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
ytrain_2 = keras.utils.to_categorical(y_train)
y_test_2 = keras.utils.to_categorical(y_test)

In [24]:
import keras

NUM_CLASSES = 2

# import necessary building blocks
from keras.models import Sequential
from keras.layers import Conv1D,  Flatten, Dense, Activation, Dropout,BatchNormalization, MaxPool1D
from keras.layers.advanced_activations import LeakyReLU

In [35]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()
    
    model.add(Conv1D(input_shape=X_train[0].shape,padding="same",kernel_size=3,filters=16))
    model.add(MaxPool1D())
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())


    
    
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())


    model.add(Dropout(0.25))
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(MaxPool1D())
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())

    model.add(Conv1D(padding="same",kernel_size=3,filters=64))
    model.add(MaxPool1D())
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())

    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(LeakyReLU(0.1))
    
    model.add(Activation("sigmoid"))
    
    return model

In [36]:
model = make_model()
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_33 (Conv1D)           (None, 1000, 16)          3136      
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 500, 16)           0         
_________________________________________________________________
leaky_re_lu_49 (LeakyReLU)   (None, 500, 16)           0         
_________________________________________________________________
batch_normalization_33 (Batc (None, 500, 16)           64        
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 500, 32)           1568      
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 500, 32)           0         
_________________________________________________________________
batch_normalization_34 (Batc (None, 500, 32)          

In [52]:
from keras import backend as K
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 25


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 2-way classification
    optimizer=keras.optimizers.Adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

# fit model
history = model.fit(
    X_train, ytrain_2,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory()],
    validation_data=(X_test, y_test_2),
    shuffle=True,
    verbose=1
)

Train on 5334 samples, validate on 2628 samples
Epoch 1/25
('Learning rate:', 0.005)
5334/5334 [==============================] - 36s 7ms/step - loss: 0.3759 - accuracy: 0.8750 - val_loss: 0.9979 - val_accuracy: 0.1195
Epoch 2/25
('Learning rate:', 0.0045)
5334/5334 [==============================] - 37s 7ms/step - loss: 0.3379 - accuracy: 0.8783 - val_loss: 0.5331 - val_accuracy: 0.8870
Epoch 3/25
('Learning rate:', 0.00405)
5334/5334 [==============================] - 38s 7ms/step - loss: 0.3299 - accuracy: 0.8783 - val_loss: 2.7755 - val_accuracy: 0.1195
Epoch 4/25
('Learning rate:', 0.003645)
5334/5334 [==============================] - 41s 8ms/step - loss: 0.3138 - accuracy: 0.8796 - val_loss: 3.5350 - val_accuracy: 0.1202
Epoch 5/25
('Learning rate:', 0.0032805)
5334/5334 [==============================] - 34s 6ms/step - loss: 0.3089 - accuracy: 0.8796 - val_loss: 0.3300 - val_accuracy: 0.8805
Epoch 6/25
('Learning rate:', 0.00295245)
5334/5334 [==============================] - 

In [53]:
test_predictions = model.predict(X_test).argmax(axis=-1)
test_answers = y_test_2.argmax(axis=-1)
test_accuracy = np.mean(test_predictions==test_answers)
print(str(test_accuracy*100)+"%")

89.95433789954338%


In [44]:
#Dataset 2 https://github.com/topazape/LSTM_Chem

text_file = open("dataset_cleansed.smi", "r")
lines = text_file.read().splitlines()
smiles2=lines

valid_mols = []
for smi in smiles2:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        valid_mols.append(mol)

In [45]:
len(valid_mols)

438540

In [46]:
len(smiles2)

438552

In [47]:
#Taking first 50000 molecules of the dataset and converting them to sdf using OpenBabel
with open('smiles.smi', 'w') as filehandle:
    for listitem in smiles2[0:50000]:
        filehandle.write('%s\n' % listitem)
        
        
!babel -ismi "smiles.smi" -osdf "smiles.sdf"

*** Open Babel Warning  in WriteMolecule
  No 2D or 3D coordinates exist. Stereochemical information will be stored using an Open Babel extension. To generate 2D or 3D coordinates instead use --gen2D or --gen3D.
*** Open Babel Warning  in ParseComplex
  Illegal aromatic element te
50000 molecules converted
50001 warnings 63 info messages 1177020 audit log messages 


In [48]:
#Calculate Molecular Fingerprint(Morgan) from iwatobipen's website.

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import DataStructs


mols = [mol for mol in Chem.SDMolSupplier('smiles.sdf') if mol != None]
for mol in mols:
    AllChem.Compute2DCoords(mol)
X = []
for mol in mols:
    arr = np.zeros((0,))
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    DataStructs.ConvertToNumpyArray(fp, arr)
    X.append(arr)

In [ ]:
#Reduce dimensions with TSNE and Tanimoto distance metric
from sklearn.manifold import TSNE

def tanimoto_dist(a,b):
    dotprod = np.dot(a,b)
    tc = dotprod / (np.sum(a) + np.sum(b) - dotprod)
    return 1.0-tc

tsne = TSNE(n_components=2, metric=tanimoto_dist)
tsne_X = tsne.fit_transform(X)
print(tsne_X.shape)

In [ ]:
plt.scatter(tsne_X.T[0], tsne_X.T[1], color='b')

In [ ]:
#Similarity search using K-nearest neighbors, use n=10 neighbors

from sklearn.neighbors import NearestNeighbors
import numpy as np

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree',metric=tanimoto_dist).fit(X)
distances, indices = nbrs.kneighbors(X)